In [4]:
import numpy as np
from pulp import *
import time
def CFLP(h:np.ndarray, c: np.ndarray, f: np.ndarray, v: np.ndarray):
    """
    Capacitated Facility Location Problem

    Parameters
    ----------

    :param h: annual demand of each customer i (i=1,...,n)
    :param c: cost to transport one unit of goods from each facility j (j=1,...,m) to each customer i
    :param f: cost to build a facility j (j=1,...,m)
    :param v: maximum capacity of each facility j (j=1,...,m)

    Decision Variables
    ------------------

    x_i = 1 if facility i is open, 0 otherwise
    y_ij = fraction of demand of customer i satisfied by facility j
    
    Objective Function
    ------------------
    min sum(j=1 to m) f_j * x_j + sum(i=1 to n) sum(j=1 to m) h_i * c_ij * y_ij
    
    Constraints
    -----------
    sum(j=1 to m) y_ij = 1 for each customer i
    y_ij <= x_j for each customer i and facility j
    sum(i=1 to n) h_i * y_ij <= v_j for each facility j
    x_j in {0,1} for each facility j
    y_ij >= 0 for each customer i and facility j
    sum(j=1 to m) v_j * x_j >= sum(i=1 to n) h_i

    Returns
    -------
    x: array of shape (m,) representing the number of facilities to open
    y: array of shape (n,m) representing the fraction of demand of each customer satisfied by each facility
    """

    m, n = c.shape
    x = LpVariable.dicts("x", range(m), cat="Binary")
    y = LpVariable.dicts("y", (range(n), range(m)), lowBound=0, upBound=1)
    prob = LpProblem("CFLP", LpMinimize)

    prob += lpSum([f[j] * x[j] for j in range(m)]) + lpSum([h[i] * c[j][i] * y[i][j] for i in range(n) for j in range(m)])

    for i in range(n):
        prob += lpSum([y[i][j] for j in range(m)]) == 1

    for i in range(n):
        for j in range(m):
            prob += y[i][j] <= x[j]

    for j in range(m):
        prob += lpSum([h[i] * y[i][j] for i in range(n)]) <= v[j]

    prob += lpSum([v[j] * x[j] for j in range(m)]) >= lpSum([h[i] for i in range(n)])

    prob.solve()

    x_sol = np.array([x[j].varValue for j in range(m)])
    y_sol = np.array([[y[i][j].varValue for j in range(m)] for i in range(n)])

    return x_sol, y_sol, value(prob.objective)

def main():
    # aleatory data of n = 100 customers and m = 100 facilities
    n, m = 1123, 1123
    h = np.random.randint(1, 100, n)
    c = np.random.randint(1, 100, (m, n))
    f = np.random.randint(1, 100, m)
    v = np.random.randint(1, 100, m)
    x, y, obj = CFLP(h, c, f, v)
    print(obj)
    print(x)
    print(y)

if __name__ == "__main__":
    main()
    


93232.00000153303
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.